In [0]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [43]:
Map = emap.Map(center=[40,-100], zoom=4)
Map.add_basemap('ROADMAP')
Map

Tiempo1 = ee.ImageCollection ('COPERNICUS/S2') \
  .filterDate ('2017-01-01' ,'2018-01-01') \
  .sort('CLOUD_COVER')
Tiempo1b = Tiempo1.reduce(ee.Reducer.median())

Tiempo2 = ee.ImageCollection ('COPERNICUS/S2') \
  .filterDate ('2018-01-01' ,'2019-01-01') \
  .sort('CLOUD_COVER');
Tiempo2b = Tiempo2.reduce(ee.Reducer.median())

NDVI1 = Tiempo1b.normalizedDifference (['B8_median', 'B4_median'])
NDVI2 = Tiempo2b.normalizedDifference (['B8_median', 'B4_median'])

# Compute the difference in NDVI.
ndviDifference = NDVI2.subtract(NDVI1)

antioquia = ee.FeatureCollection('users/evaristizabalg/Antioquia')

# Update the NDVI difference mask with the land mask.
maskedDifference = ndviDifference.updateMask(landMask).clip(antioquia)

# Display the masked result.
vizParams = {'min': -0.5, 'max': 0.5,
             'palette': ['FF0000', 'FFFFFF', '0000FF']}
Map.center_object(antioquia,7)
Map.addLayer(maskedDifference, vizParams, 'NDVI difference')

Map.addLayerControl()
Map